In [72]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [73]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [74]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [75]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [76]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet152(pretrained='imagenet')
num_ftrs = model_conv.classifier.in_features
maxm = 0
for i, param in model_conv.named_parameters():
    maxm = maxm + 1
    if maxm < 182:
      param.requires_grad = False
print(maxm)
model_conv.classifier = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to C:\Users\rohit/.torch\models\resnet152-b121ed2d.pth



0it [00:00, ?it/s]


8192it [00:00, 39548.14it/s]


40960it [00:00, 50925.41it/s]


106496it [00:00, 68215.27it/s]


245760it [00:00, 93346.85it/s]


524288it [00:01, 129616.26it/s]


909312it [00:01, 182151.93it/s]


1081344it [00:01, 247024.98it/s]


1581056it [00:01, 345563.58it/s]


1982464it [00:01, 473056.86it/s]


2473984it [00:01, 648831.51it/s]


2850816it [00:01, 772580.66it/s]


3522560it [00:01, 1043403.29it/s]


3932160it [00:02, 1340619.76it/s]


4407296it [00:02, 1707880.21it/s]


4816896it [00:02, 1974041.22it/s]


5283840it [00:02, 2338467.61it/s]


5783552it [00:02, 2771368.82it/s]


6201344it [00:02, 2953132.78it/s]


6602752it [00:02, 2513341.75it/s]


7045120it [00:02, 2874909.46it/s]


7413760it [00:03, 2659524.14it/s]


7839744it [00:03, 2984766.05it/s]


8216576it [00:03, 3138446.91it/s]


8716288it [00:03, 3407058.00it/s]


9175040

135118848it [01:04, 2657999.66it/s]


135397376it [01:04, 2684936.33it/s]


135675904it [01:04, 2633258.39it/s]


136011776it [01:04, 2793376.00it/s]


136298496it [01:04, 2760760.17it/s]


136642560it [01:04, 2920957.38it/s]


136978432it [01:04, 2907404.95it/s]


137322496it [01:05, 2849871.50it/s]


137650176it [01:05, 2933164.98it/s]


137953280it [01:05, 1938423.42it/s]


138346496it [01:05, 2261526.08it/s]


138649600it [01:05, 2447853.61it/s]


138944512it [01:05, 2578788.53it/s]


139239424it [01:05, 2647884.61it/s]


139534336it [01:05, 2723373.65it/s]


139853824it [01:06, 2821079.70it/s]


140189696it [01:06, 2951960.95it/s]


140500992it [01:06, 2630480.52it/s]


140787712it [01:06, 2591722.23it/s]


141164544it [01:06, 2822078.70it/s]


141475840it [01:06, 2893820.60it/s]


141778944it [01:06, 2885825.60it/s]


142196736it [01:06, 3166759.58it/s]


142532608it [01:06, 3209384.28it/s]


142901248it [01:07, 3307721.78it/s]


143245312it [01:07, 2861623.41it/s]


143548416it 

AttributeError: 'ResNet' object has no attribute 'classifier'

In [17]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)




In [24]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [25]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=30)

Epoch 0/29
----------





  0%|                                                                                          | 0/453 [00:00<?, ?it/s]


  0%|▏                                                                                 | 1/453 [00:00<02:25,  3.10it/s]


  0%|▎                                                                                 | 2/453 [00:00<02:20,  3.20it/s]


  1%|▌                                                                                 | 3/453 [00:00<02:15,  3.32it/s]


  1%|▋                                                                                 | 4/453 [00:01<02:11,  3.42it/s]


  1%|▉                                                                                 | 5/453 [00:01<02:09,  3.46it/s]


  1%|█                                                                                 | 6/453 [00:01<02:06,  3.52it/s]


  2%|█▎                                                                                | 7/453 [00:01<02:04,  3.57it/s]


  2%|█▍              

 30%|███████████████████████▋                                                        | 134/453 [00:38<01:39,  3.22it/s]


 30%|███████████████████████▊                                                        | 135/453 [00:38<01:38,  3.24it/s]


 30%|████████████████████████                                                        | 136/453 [00:38<01:34,  3.35it/s]


 30%|████████████████████████▏                                                       | 137/453 [00:39<01:34,  3.33it/s]


 30%|████████████████████████▎                                                       | 138/453 [00:39<01:32,  3.41it/s]


 31%|████████████████████████▌                                                       | 139/453 [00:39<01:30,  3.48it/s]


 31%|████████████████████████▋                                                       | 140/453 [00:39<01:28,  3.54it/s]


 31%|████████████████████████▉                                                       | 141/453 [00:40<01:27,  3.56it/s]


 31%|███████████████████

 59%|███████████████████████████████████████████████▎                                | 268/453 [01:17<00:57,  3.20it/s]


 59%|███████████████████████████████████████████████▌                                | 269/453 [01:17<00:59,  3.08it/s]


 60%|███████████████████████████████████████████████▋                                | 270/453 [01:17<01:00,  3.01it/s]


 60%|███████████████████████████████████████████████▊                                | 271/453 [01:18<01:01,  2.97it/s]


 60%|████████████████████████████████████████████████                                | 272/453 [01:18<00:58,  3.11it/s]


 60%|████████████████████████████████████████████████▏                               | 273/453 [01:18<00:55,  3.22it/s]


 60%|████████████████████████████████████████████████▍                               | 274/453 [01:19<00:54,  3.30it/s]


 61%|████████████████████████████████████████████████▌                               | 275/453 [01:19<00:52,  3.39it/s]


 61%|███████████████████

 89%|██████████████████████████████████████████████████████████████████████▉         | 402/453 [01:57<00:16,  3.12it/s]


 89%|███████████████████████████████████████████████████████████████████████▏        | 403/453 [01:57<00:15,  3.24it/s]


 89%|███████████████████████████████████████████████████████████████████████▎        | 404/453 [01:57<00:14,  3.34it/s]


 89%|███████████████████████████████████████████████████████████████████████▌        | 405/453 [01:58<00:14,  3.40it/s]


 90%|███████████████████████████████████████████████████████████████████████▋        | 406/453 [01:58<00:13,  3.43it/s]


 90%|███████████████████████████████████████████████████████████████████████▉        | 407/453 [01:58<00:13,  3.45it/s]


 90%|████████████████████████████████████████████████████████████████████████        | 408/453 [01:58<00:13,  3.45it/s]


 90%|████████████████████████████████████████████████████████████████████████▏       | 409/453 [01:59<00:12,  3.45it/s]


 91%|███████████████████

...

 59%|███████████████████████████████████████████████▎                                | 268/453 [01:15<00:52,  3.51it/s]


 59%|███████████████████████████████████████████████▌                                | 269/453 [01:15<00:52,  3.53it/s]


 60%|███████████████████████████████████████████████▋                                | 270/453 [01:15<00:51,  3.58it/s]


 60%|███████████████████████████████████████████████▊                                | 271/453 [01:16<00:50,  3.57it/s]


 60%|████████████████████████████████████████████████                                | 272/453 [01:16<00:49,  3.63it/s]


 60%|████████████████████████████████████████████████▏                               | 273/453 [01:16<00:49,  3.65it/s]


 60%|████████████████████████████████████████████████▍                               | 274/453 [01:16<00:49,  3.61it/s]


 61%|████████████████████████████████████████████████▌                               | 275/453 [01:17<00:49,  3.59it/s]


 61%|███████████████████

 89%|██████████████████████████████████████████████████████████████████████▉         | 402/453 [01:53<00:14,  3.61it/s]


 89%|███████████████████████████████████████████████████████████████████████▏        | 403/453 [01:53<00:13,  3.66it/s]


 89%|███████████████████████████████████████████████████████████████████████▎        | 404/453 [01:53<00:13,  3.58it/s]


 89%|███████████████████████████████████████████████████████████████████████▌        | 405/453 [01:53<00:13,  3.64it/s]


 90%|███████████████████████████████████████████████████████████████████████▋        | 406/453 [01:54<00:12,  3.66it/s]


 90%|███████████████████████████████████████████████████████████████████████▉        | 407/453 [01:54<00:12,  3.65it/s]


 90%|████████████████████████████████████████████████████████████████████████        | 408/453 [01:54<00:12,  3.61it/s]


 90%|████████████████████████████████████████████████████████████████████████▏       | 409/453 [01:55<00:12,  3.62it/s]


 91%|███████████████████

phase Loss: 0.0001 Acc: 0.9985





  0%|                                                                                           | 0/80 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/80 [00:00<00:11,  6.87it/s]


  2%|██                                                                                 | 2/80 [00:00<00:11,  6.77it/s]


  4%|███                                                                                | 3/80 [00:00<00:11,  6.42it/s]


  5%|████▏                                                                              | 4/80 [00:00<00:11,  6.47it/s]


  6%|█████▏                                                                             | 5/80 [00:00<00:11,  6.47it/s]


  8%|██████▏                                                                            | 6/80 [00:00<00:11,  6.34it/s]


  9%|███████▎                                                                           | 7/80 [00:01<00:11,  6.55it/s]


 10%|████████▎       

phase Loss: 0.0088 Acc: 0.9429
7.8125e-06
Training complete in 70m 7s
Best val Acc: 0.953425


In [26]:
def test_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs, labels_ = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        labels.append(labels_)

    return predictions, labels
  
predictions, labels = test_model(model_conv, val_loader, optimizer_conv, True)




  0%|                                                                                           | 0/80 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/80 [00:00<00:12,  6.15it/s]


  2%|██                                                                                 | 2/80 [00:00<00:12,  6.37it/s]


  4%|███                                                                                | 3/80 [00:00<00:12,  6.32it/s]


  5%|████▏                                                                              | 4/80 [00:00<00:11,  6.45it/s]


  6%|█████▏                                                                             | 5/80 [00:00<00:11,  6.57it/s]


  8%|██████▏                                                                            | 6/80 [00:00<00:10,  6.74it/s]


  9%|███████▎                                                                           | 7/80 [00:01<00:10,  6.67it/s]


 10%|████████▎       

In [67]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'densenet121.pth')

In [39]:
predictions1, labels1 = torch.cat(predictions), torch.cat(labels)
predictions1, labels1 = np.array(predictions1.cpu()), np.array(labels1)

In [40]:
predictions1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [41]:
labels1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [42]:
len(labels1)

2555

In [48]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [49]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)




  0%|                                                                                          | 0/229 [00:00<?, ?it/s]


  0%|▎                                                                                 | 1/229 [00:00<00:34,  6.55it/s]


  1%|▋                                                                                 | 2/229 [00:00<00:35,  6.40it/s]


  1%|█                                                                                 | 3/229 [00:00<00:35,  6.42it/s]


  2%|█▍                                                                                | 4/229 [00:00<00:34,  6.51it/s]


  2%|█▊                                                                                | 5/229 [00:00<00:36,  6.15it/s]


  3%|██▏                                                                               | 6/229 [00:01<00:38,  5.73it/s]


  3%|██▌                                                                               | 7/229 [00:01<00:40,  5.52it/s]


  3%|██▊             

 59%|██████████████████████████████████████████████▊                                 | 134/229 [00:20<00:14,  6.53it/s]


 59%|███████████████████████████████████████████████▏                                | 135/229 [00:20<00:14,  6.53it/s]


 59%|███████████████████████████████████████████████▌                                | 136/229 [00:21<00:14,  6.53it/s]


 60%|███████████████████████████████████████████████▊                                | 137/229 [00:21<00:14,  6.51it/s]


 60%|████████████████████████████████████████████████▏                               | 138/229 [00:21<00:13,  6.54it/s]


 61%|████████████████████████████████████████████████▌                               | 139/229 [00:21<00:13,  6.54it/s]


 61%|████████████████████████████████████████████████▉                               | 140/229 [00:21<00:13,  6.52it/s]


 62%|█████████████████████████████████████████████████▎                              | 141/229 [00:21<00:13,  6.52it/s]


 62%|███████████████████

In [52]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [54]:
len(predictions3)

7301

In [59]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [57]:
result = pd.read_csv('resnet50_15_col.csv')

In [61]:
result.shape[0]

7301

In [62]:
result['label'] = predictions3

In [63]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,2
4,14.jpg,5


In [64]:
result.to_csv('torch_densenet121.csv')

# ResNet101

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [3]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [4]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [5]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet101(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


In [7]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 135:
        param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [8]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [11]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs), Variable(labels)
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [13]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=25)

Epoch 0/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [03:21<00:00,  2.58it/s]


phase Loss: 0.0058 Acc: 0.9333


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  5.90it/s]


phase Loss: 0.0080 Acc: 0.9205
0.0005
Epoch 1/24
----------


...

phase Loss: 0.0101 Acc: 0.9378
7.8125e-06
Epoch 24/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [03:21<00:00,  2.63it/s]


phase Loss: 0.0002 Acc: 0.9982


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  5.88it/s]


phase Loss: 0.0108 Acc: 0.9386
7.8125e-06
Training complete in 90m 3s
Best val Acc: 0.940900


In [14]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet101.pth')

In [15]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [16]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:35<00:00,  4.73it/s]


In [17]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [18]:
len(predictions3)

7301

In [19]:
predictions3[:5]

array([5, 0, 4, 4, 5], dtype=int64)

In [20]:
result = pd.read_csv('resnet50_15_col.csv')

In [21]:
result.shape[0]

7301

In [22]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [24]:
result.to_csv('torch_resnet101.csv')

# ResNet152

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [3]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [4]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [5]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet152(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


In [11]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
print(maxm)

467


In [12]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 445:
        param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [13]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [16]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [18]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=5)

Epoch 0/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:54<00:00,  3.94it/s]


phase Loss: 0.0053 Acc: 0.9391


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.59it/s]


phase Loss: 0.0078 Acc: 0.9162
0.0005
Epoch 1/4
----------


...

phase Loss: 0.0071 Acc: 0.9284
0.0005
Epoch 4/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:56<00:00,  3.91it/s]


phase Loss: 0.0040 Acc: 0.9543


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.58it/s]


phase Loss: 0.0068 Acc: 0.9319
0.0005
Training complete in 11m 5s
Best val Acc: 0.931898


In [19]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet152.pth')

In [20]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [21]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:48<00:00,  4.56it/s]


In [22]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [23]:
len(predictions3)

7301

In [24]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [25]:
result = pd.read_csv('resnet50_15_col.csv')

In [26]:
result.shape[0]

7301

In [27]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [28]:
result.to_csv('torch_resnet152.csv')

# Densenet169

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [3]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [4]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [5]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet152(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


In [11]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
print(maxm)

467


In [12]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 445:
        param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [13]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [16]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [18]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=5)

Epoch 0/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:54<00:00,  3.94it/s]


phase Loss: 0.0053 Acc: 0.9391


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.59it/s]


phase Loss: 0.0078 Acc: 0.9162
0.0005
Epoch 1/4
----------


...

phase Loss: 0.0071 Acc: 0.9284
0.0005
Epoch 4/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:56<00:00,  3.91it/s]


phase Loss: 0.0040 Acc: 0.9543


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.58it/s]


phase Loss: 0.0068 Acc: 0.9319
0.0005
Training complete in 11m 5s
Best val Acc: 0.931898


In [19]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet152.pth')

In [20]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [21]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:48<00:00,  4.56it/s]


In [22]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [23]:
len(predictions3)

7301

In [24]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [25]:
result = pd.read_csv('resnet50_15_col.csv')

In [26]:
result.shape[0]

7301

In [27]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [28]:
result.to_csv('torch_resnet152.csv')